# 数据预处理

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
import sys
import scipy as sp
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

In [2]:
#训练集用户特征数据
train_agg = pd.read_csv('train/train_agg.csv',sep='\t')
#训练集用户APP操作日志
train_log = pd.read_csv('train/train_log.csv',sep='\t',parse_dates = ['OCC_TIM'])
#训练集用户标签
train_flg = pd.read_csv('train/train_flg.csv',sep='\t')

### 用户特征数据与标签合并处理

In [3]:
all_train = pd.merge(train_flg,train_agg,on=['USRID'],how='left')
all_train[1:10]

,USRID,FLAG,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
1,35,0,0.25427,1.10266,2.10801,0.28920,-0.68454,-0.63080,-0.29641,-0.18761,...,-0.22881,0.15815,-0.16201,0.47620,-0.55917,-0.77398,-0.25708,-0.57910,-0.47456,-0.32438
2,42,0,-0.36853,-0.90689,-0.20076,0.28920,1.46083,-0.76522,-0.29641,-0.18761,...,0.55795,3.90892,-0.16201,-0.11429,-0.03534,-0.26368,-0.25708,0.96800,0.52019,0.88724
3,76,0,0.64353,-0.90689,-0.20076,0.28920,-0.68454,-0.22753,6.09153,0.00731,...,-0.22881,0.15815,-0.16201,0.47620,-0.55917,-0.77398,-0.25708,-0.53975,-0.51462,-0.32438
4,77,0,-0.47233,-0.90689,-0.37835,0.28920,1.46083,-0.63080,-0.29641,-0.18761,...,-0.22881,-0.77954,-0.16201,0.14203,0.67834,0.24662,-0.25708,0.45938,-0.47425,-0.32438
5,96,0,-0.34258,1.10266,2.10801,0.28920,-0.68454,-1.03407,0.03980,-0.18552,...,-0.22881,0.15815,-0.16201,-0.28935,-0.54656,0.24662,0.19588,-0.49022,-0.10965,-0.32438
6,125,0,-0.52423,-0.90689,-0.20076,-3.45785,-0.68454,0.57901,-0.29641,-0.18761,...,-0.22881,0.15815,-0.16201,0.25700,-0.55917,1.26722,-0.25708,-0.60930,-0.53843,-0.32438
7,136,0,-0.57613,1.10266,-0.37835,0.28920,-0.68454,-1.30291,-0.29641,-0.18761,...,-0.20290,0.15815,-0.16201,0.47620,-0.07078,-0.77398,-0.23052,-0.16827,-0.54884,0.07949
8,146,0,0.07262,1.10266,-0.37835,0.28920,-0.68454,0.04132,-0.29641,-0.18761,...,-0.22881,0.15815,-0.16201,0.45226,-0.53708,-0.26368,-0.25385,-0.48677,-0.27066,-0.32438
9,190,0,0.07262,1.10266,-0.37835,0.28920,-0.68454,1.38555,-0.29641,-0.18761,...,-0.22881,0.15815,4.50312,-1.35926,1.31311,0.75692,-0.25708,0.14524,-0.54884,-0.32438


### 用户一个月APP操作日志数据处理与特征提取

In [4]:
#日志读取
log = train_log
log['EVT_LBL_1'] = log['EVT_LBL'].apply(lambda x:x.split('-')[0])
log['EVT_LBL_2'] = log['EVT_LBL'].apply(lambda x:x.split('-')[1])
log['EVT_LBL_3'] = log['EVT_LBL'].apply(lambda x:x.split('-')[2])
log[1:10]

,USRID,EVT_LBL,OCC_TIM,TCH_TYP,EVT_LBL_1,EVT_LBL_2,EVT_LBL_3
1,10002,163-578-914,2018-03-22 16:31:18,0,163,578,914
2,10002,259-924-1525,2018-03-22 16:31:15,0,259,924,1525
3,10002,326-1040-1677,2018-03-06 12:08:51,0,326,1040,1677
4,10002,326-1041-1678,2018-03-09 14:40:22,0,326,1041,1678
5,10002,326-1041-1678,2018-03-16 17:45:42,0,326,1041,1678
6,10002,359-1234-2004,2018-03-15 10:40:35,0,359,1234,2004
7,10002,520-1836-3640,2018-03-06 12:07:02,0,520,1836,3640
8,10002,520-1836-3640,2018-03-09 11:23:01,0,520,1836,3640
9,10002,520-1836-3640,2018-03-22 16:31:13,0,520,1836,3640


In [5]:
#用户操作日志中，统计每个用户操作频率
EVT_LBL_len = train_log.groupby(by= ['USRID'], as_index = False)['EVT_LBL'].agg({'EVT_LBL_len':len})
EVT_LBL_len[1:10]

,USRID,EVT_LBL_len
1,3,157
2,4,18
3,5,20
4,7,76
5,11,126
6,12,53
7,13,104
8,16,200
9,19,9


In [6]:
#统计所有用户进入某个APP页面的次数
from collections import Counter
import operator
ls = []
ls = train_log['EVT_LBL'].apply(lambda x:x.split('-')[2])
fs = Counter(ls)
sorted(fs.items(),key=operator.itemgetter(1))[-10:]

[('1724', 102041),
 ('1703', 113888),
 ('1525', 129805),
 ('1677', 140601),
 ('1523', 186941),
 ('117', 239437),
 ('2004', 257231),
 ('277', 286701),
 ('3640', 565582),
 ('1678', 761883)]

In [7]:
#操作日志中用户数目
all_list= np.unique(log['USRID'])
all_list

array([    2,     3,     4, ..., 99995, 99996, 99998], dtype=int64)

In [8]:
#计算日志中每个用户进入某个页面的次数
numls = ['1678','3640','277'] #用户访问次数最多的前三行为
def all_count_EVT_LBL_0(all_list, log,x):
    countls=[]
    for i in range(0,len(all_list)):
        tmp = log[log.USRID==all_list[i]]
        count = 0
        for j in range(1,len(tmp)):
            if tmp.EVT_LBL_3.values[j]== x:
                count = count + 1
        countls.append(count)
    return countls

all_USRID_count2= DataFrame(all_list)
for i in range(0,len(numls)):
    all_count = all_count_EVT_LBL_0(all_list, log,numls[i])
    all_USRID_count2[numls[i]]=DataFrame(all_count)

In [9]:
log_EVT_LBL = all_USRID_count2
log_EVT_LBL

,0,1678,3640,277
0,2,3,1,0
1,3,37,25,11
2,4,6,3,1
3,5,6,3,2
4,7,25,6,6
5,11,26,28,10
6,12,6,11,3
7,13,25,9,5
8,16,44,25,15
9,19,2,2,1


### 所有数据合并

In [10]:
train_agg = pd.read_csv('train/train_agg.csv',sep='\t')
train_flg = pd.read_csv('train/train_flg.csv',sep='\t')
train_log = pd.read_csv('train/train_log.csv',sep='\t',parse_dates = ['OCC_TIM'])
EVT_LBL_len = train_log.groupby(by= ['USRID'], as_index = False)['EVT_LBL'].agg({'EVT_LBL_len':len})
EVT_LBL_set_len = train_log.groupby(by= ['USRID'], as_index = False)['EVT_LBL'].agg({'EVT_LBL_set_len':lambda x:len(set(x))})
tmp = log_EVT_LBL[0]
log_EVT_LBL['USRID']=tmp
all_train = pd.merge(train_flg,train_agg,on=['USRID'],how='left')
all_train = pd.merge(all_train,EVT_LBL_len,on=['USRID'],how='left')
all_train = pd.merge(all_train,EVT_LBL_set_len,on=['USRID'],how='left')
all_train = pd.merge(all_train,log_EVT_LBL,on=['USRID'],how='left')
all_train = all_train.drop([0], axis=1)
all_train.fillna(0,inplace=True)

In [11]:
all_train[1:10]

,USRID,FLAG,V1,V2,V3,V4,V5,V6,V7,V8,...,V26,V27,V28,V29,V30,EVT_LBL_len,EVT_LBL_set_len,1678,3640,277
1,35,0,0.25427,1.10266,2.10801,0.28920,-0.68454,-0.63080,-0.29641,-0.18761,...,-0.77398,-0.25708,-0.57910,-0.47456,-0.32438,33.0,13.0,1.0,12.0,0.0
2,42,0,-0.36853,-0.90689,-0.20076,0.28920,1.46083,-0.76522,-0.29641,-0.18761,...,-0.26368,-0.25708,0.96800,0.52019,0.88724,65.0,9.0,15.0,9.0,7.0
3,76,0,0.64353,-0.90689,-0.20076,0.28920,-0.68454,-0.22753,6.09153,0.00731,...,-0.77398,-0.25708,-0.53975,-0.51462,-0.32438,685.0,38.0,83.0,99.0,24.0
4,77,0,-0.47233,-0.90689,-0.37835,0.28920,1.46083,-0.63080,-0.29641,-0.18761,...,0.24662,-0.25708,0.45938,-0.47425,-0.32438,0.0,0.0,0.0,0.0,0.0
5,96,0,-0.34258,1.10266,2.10801,0.28920,-0.68454,-1.03407,0.03980,-0.18552,...,0.24662,0.19588,-0.49022,-0.10965,-0.32438,107.0,35.0,13.0,35.0,3.0
6,125,0,-0.52423,-0.90689,-0.20076,-3.45785,-0.68454,0.57901,-0.29641,-0.18761,...,1.26722,-0.25708,-0.60930,-0.53843,-0.32438,102.0,20.0,26.0,12.0,10.0
7,136,0,-0.57613,1.10266,-0.37835,0.28920,-0.68454,-1.30291,-0.29641,-0.18761,...,-0.77398,-0.23052,-0.16827,-0.54884,0.07949,0.0,0.0,0.0,0.0,0.0
8,146,0,0.07262,1.10266,-0.37835,0.28920,-0.68454,0.04132,-0.29641,-0.18761,...,-0.26368,-0.25385,-0.48677,-0.27066,-0.32438,35.0,14.0,11.0,3.0,2.0
9,190,0,0.07262,1.10266,-0.37835,0.28920,-0.68454,1.38555,-0.29641,-0.18761,...,0.75692,-0.25708,0.14524,-0.54884,-0.32438,0.0,0.0,0.0,0.0,0.0


In [12]:
all_train.to_csv('all_train.csv',index=None,sep='\t')

In [13]:
#测试数据
test_agg = pd.read_csv('test/test_agg.csv',sep='\t')
test_log = pd.read_csv('test/test_log.csv',sep='\t',parse_dates = ['OCC_TIM'])

def log_tabel(data):
    EVT_LBL_len = data.groupby(by= ['USRID'], as_index = False)['EVT_LBL'].agg({'EVT_LBL_len':len})
    EVT_LBL_set_len = data.groupby(by= ['USRID'], as_index = False)['EVT_LBL'].agg({'EVT_LBL_set_len':lambda x:len(set(x))})
    return EVT_LBL_len,EVT_LBL_set_len
    
EVT_LBL_len,EVT_LBL_set_len = log_tabel(test_log)

log = test_log
log['EVT_LBL_1'] = log['EVT_LBL'].apply(lambda x:x.split('-')[0])
log['EVT_LBL_2'] = log['EVT_LBL'].apply(lambda x:x.split('-')[1])
log['EVT_LBL_3'] = log['EVT_LBL'].apply(lambda x:x.split('-')[2])
all_list= np.unique(log['USRID'])

numls = ['1678','3640','277']
def all_count_EVT_LBL_0(all_list, log,x):
    countls=[]
    for i in range(0,len(all_list)):
        tmp = log[log.USRID==all_list[i]]
        count = 0
        for j in range(1,len(tmp)):
            if tmp.EVT_LBL_3.values[j]== x:
                count = count + 1
        countls.append(count)
    return countls

all_USRID_count2= DataFrame(all_list)
for i in range(0,len(numls)):
    all_count = all_count_EVT_LBL_0(all_list, log,numls[i])
    all_USRID_count2[numls[i]]=DataFrame(all_count)

tmp = all_USRID_count2[0]
all_USRID_count2['USRID']=tmp

test_set = pd.merge(test_agg,EVT_LBL_len,on=['USRID'],how='left')
test_set = pd.merge(test_set,EVT_LBL_set_len,on=['USRID'],how='left')
test_set = pd.merge(test_set,all_USRID_count2,on=['USRID'],how='left')
test_set = test_set.drop([0], axis=1)
test_set.fillna(0,inplace=True)


In [14]:
test_set[1:10]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V27,V28,V29,V30,USRID,EVT_LBL_len,EVT_LBL_set_len,1678,3640,277
1,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,-1.03407,0.03980,-0.13652,-0.48351,-0.55402,...,-0.25708,-0.42709,-0.54710,-0.32438,4572,0.0,0.0,0.0,0.0,0.0
2,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,-0.09311,-0.29641,-0.18761,-0.33740,-0.46000,...,-0.25708,-0.40814,-0.53054,-0.32438,64470,105.0,14.0,27.0,16.0,13.0
3,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,0.04132,-0.29641,-0.18761,-0.50786,-0.60103,...,-0.25708,-0.57910,-0.54868,-0.32438,68958,49.0,17.0,13.0,12.0,3.0
4,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,0.04132,-0.29641,-0.18761,-0.48351,-0.53051,...,-0.25708,-0.16073,-0.54884,0.07949,8839,1.0,1.0,0.0,0.0,0.0
5,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,0.44459,-0.29641,-0.18761,-0.48351,-0.48350,...,-0.25708,-0.53440,-0.54332,0.88724,15106,0.0,0.0,0.0,0.0,0.0
6,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,0.98228,-0.29641,-0.18761,-0.48351,-0.57752,...,-0.25708,-0.57020,-0.54789,-0.32438,28227,0.0,0.0,0.0,0.0,0.0
7,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,2.86420,-0.29641,-0.18761,-0.33740,-0.38949,...,-0.25708,-0.54999,-0.54647,0.07949,76977,46.0,12.0,8.0,15.0,3.0
8,-1.92554,-0.90689,-1.26634,0.2892,1.46083,-1.70618,-0.29641,-0.18761,-0.43480,-0.53051,...,-0.25708,-0.37300,-0.53338,-0.32438,87072,198.0,31.0,51.0,28.0,16.0
9,-1.92554,-0.90689,-1.26634,0.2892,1.46083,-1.57176,-0.29641,-0.18761,-0.50786,-0.60103,...,-0.25708,-0.57910,-0.54884,-0.32438,40428,0.0,0.0,0.0,0.0,0.0


In [15]:
test_set.to_csv('test_set.csv',index=None,sep='\t')